In [6]:

import pandas as pd
import pandas_datareader.data as web
from pandas import Series, DataFrame
import datetime
from datetime import date, timedelta, datetime
import math
import matplotlib as mpl
import numpy as np
import talib as ta
import yfinance as yf
from tqdm import tqdm
from os import walk
import os
import matplotlib.pyplot as plt
import random
from random import randrange
from collections import defaultdict

In [7]:
def random_date(start, end):
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)

In [8]:
def buyStock2(row):
    close_adj = row[4]
    starting_price = close_adj
    return close_adj,starting_price,True

def sellStock2(row,starting_price):
    close_adj = row[4]
    single_trade_percent_gain = ((close_adj - starting_price) / starting_price) 
    return close_adj,single_trade_percent_gain,False

In [9]:
def tradeVerison2(stock, startDate, endDate,plotVal,WindowVal):
    StartingAmount = 500_000/505
    oldDate = startDate
    startDate = startDate - timedelta(39)
    df = pd.read_csv(
        "C:/Users/rohan/Desktop/SchoolWork/StockMarketSimulationIQP/RohanCode/Data/filter_30y_stock_csvs/{}.csv".format(stock))
    df = df.rename(columns={"CLOSE": "Adj Close", "VOLUME": "Volume"})

    df['Datetime'] = pd.to_datetime(df['Datetime'])
    g_fd = df
    g_fd = g_fd[~(g_fd['Datetime'] < oldDate)]
    g_fd = g_fd[~(g_fd['Datetime'] > endDate)]
  
    df = df[~(df['Datetime'] < startDate)]
    df = df[~(df['Datetime'] > endDate)]

    StockData = pd.DataFrame(df).to_numpy()
 
    
    mapForBuy = {}
    mapForSellP = {}
    mapForSellN = {}

    gainsList = []
    lostList = []

    Total_Gain = 0
    starting_price = 1

    TotalReturn = StartingAmount
    index = 28
    bought = False
    val = StockData.shape
    for rows in StockData:
        row = StockData[index][:]
        if(index >= val[0] -1):
            break

        maxValuePeriod = df.iloc[index-WindowVal:index-1]['Adj Close'].max()

        Valuemean = df.iloc[index-WindowVal:index-1]['Adj Close'].mean()

        ValuemeanOld = df.iloc[index-(WindowVal+7):index-7]['Adj Close'].mean()
        stvalue = df.iloc[index-(WindowVal+3):index-1]['Adj Close'].std()
        

        if math.isnan(maxValuePeriod):
            maxValuePeriod = 0
        if math.isnan(Valuemean):
            Valuemean = 0

        if not bought:
            if(StockData[index-1][8] > 12):

                if StockData[index-1][11] == 1 and StockData[index-1][10] == 1:
                    close_adj, starting_price, bought = buyStock2(row)
                    mapForBuy[row[0]] = row[4]

                elif maxValuePeriod - stvalue  < Valuemean :
                        
                    close_adj, starting_price, bought = buyStock2(row)
                    mapForBuy[row[0]] = row[4]
                        
                else:
                    gainsList.append(0)

        elif bought and close_adj != row[4]:  # This is sell clause
            if (StockData[index-1][11] == 0 or StockData[index-1][9] < 41) and StockData[index-1][10] == 0 and ValuemeanOld < Valuemean:
                infoOnSelling = sellStock2(row, starting_price)
                close_adj = infoOnSelling[0]
                Total_Gain += infoOnSelling[1]
                bought = infoOnSelling[2]

                gainsList.append(infoOnSelling[1])

                if(infoOnSelling[1] >= 0):
                    mapForSellP[row[0]] = row[4]

                else:
                    lostList.append(infoOnSelling[1])
                    mapForSellN[row[0]] = row[4]

            else:
                yesterday = StockData[index-1][4]
                today = row[4]
                checker = (today - yesterday)/yesterday
                if(checker < 0):
                    lostList.append(checker)
                gainsList.append(checker)
                
        index += 1
        TotalReturn = (1 + Total_Gain) * TotalReturn
        Total_Gain = 0




    if bought:
        row = StockData[index][:]
        infoOnSelling = sellStock2(row, starting_price)
        close_adj = infoOnSelling[0]
        Total_Gain += infoOnSelling[1]
        bought = infoOnSelling[2]

        gainsList.append(infoOnSelling[1])

        if(infoOnSelling[1] >= 0):
            mapForSellP[row[0]] = row[4]

        else:
            lostList.append(infoOnSelling[1])
            mapForSellN[row[0]] = row[4]

        TotalReturn = (1 + Total_Gain) * TotalReturn
        Total_Gain = 0

    if(plotVal):
        plt.plot(g_fd['Datetime'], g_fd['Adj Close'])
        plt.scatter(mapForBuy.keys(), mapForBuy.values(), label="Buy", color="Blue",
                    marker="o", s=30)
        plt.scatter(mapForSellP.keys(), mapForSellP.values(), label="Sell - Gain", color="Green",
                    marker="^", s=30)
        plt.scatter(mapForSellN.keys(), mapForSellN.values(), label="Sell - Lost", color="Red",
                    marker="v", s=30)

        plt.xlabel("Date")
        plt.ylabel("Price $")
        plt.title("{}".format(stock))
        plt.tick_params(axis='x',which='major', labelsize=7, rotation = 90)
        #plt.tick_params(axis='y',colors='white')
        
        plt.legend()
        plt.tight_layout()

        plt.savefig('GraphsForBuySell/{}.png'.format(stock),facecolor = 'w')
        plt.close()

    tradingdays = (endDate-startDate).days
    if not mapForBuy or not np.std(gainsList):
        sharpeRatio = 0
    else:
        sharpeRatio = np.mean(gainsList)/np.std(gainsList)
        sharpeRatio = (tradingdays ** 0.5) * sharpeRatio

    if not lostList or not np.std(lostList):
        sortinoRatio = 0
    else:
        sortinoRatio = (np.mean(gainsList)/np.std(lostList)) * (tradingdays ** 0.5)
    
    return round(TotalReturn-StartingAmount, 2), sharpeRatio, sortinoRatio


In [10]:
totalWins = 0
f = []
testCases = 10
for (dirpath, dirnames, filenames) in walk("C:/Users/rohan/Desktop/SchoolWork/StockMarketSimulationIQP/RohanCode/Data/clean_30y_stock_csvs/"):
    f.extend(filenames)
    break

ROIopt = defaultdict(list)

for i in range(testCases):
    print("Starting number {} for test cases".format(i+1))
    d1 = datetime.strptime('1/1/2000', '%m/%d/%Y')
    d2 = datetime.strptime('10/1/2021', '%m/%d/%Y')
    startDate = random_date(d1, d2)
    startDate = startDate.strftime('%Y-%m-%d')
    startDate = datetime.strptime(startDate, "%Y-%m-%d")

    days = timedelta(61)
    endDate = startDate + days

    for windowVal in tqdm(range(1,26)):
        money = {}
        sum = 0

        for stock in f:
            stock = stock[:-4]
            profit, sharpeRatio, sortinoRatio = tradeVerison2(stock,startDate,endDate,False,windowVal)
            money[stock] = profit
    
        for s in money:
            sum+= money[s]
            percentageOfWin = ((sum)/500_000) * 100
        
        ROIopt[windowVal].append(percentageOfWin)

  0%|          | 0/25 [00:00<?, ?it/s]

Starting number 1 for test cases


  0%|          | 0/25 [00:00<?, ?it/s]

Starting number 2 for test cases


  0%|          | 0/25 [00:00<?, ?it/s]

...

  0%|          | 0/25 [00:00<?, ?it/s]

Starting number 9 for test cases


  0%|          | 0/25 [00:00<?, ?it/s]

Starting number 10 for test cases


100%|██████████| 25/25 [12:46<00:00, 30.66s/it]


In [11]:
# print(ROIopt.keys())
# print(ROIopt.values())
valsOfOpt = []

for val in ROIopt:
    valsOfOpt.append(np.mean(ROIopt[val]))

# print(valsOfOpt)
plt.plot(ROIopt.keys(),valsOfOpt)
plt.xlabel("Window value")
plt.ylabel("ROI %")
plt.title("Window Optimization")
plt.tight_layout()
plt.savefig('OptimizationGraph/{}.png'.format("Window Optimization"),facecolor = 'w')
plt.close()

In [12]:
max = 0
master = 0

for key in ROIopt.keys():
    if(max < np.mean(ROIopt[key])):
        max = np.mean(ROIopt[key])
        master = key

print(master)
print(max)

11
1.0691003999999995
